## Where we got the data
We got the data from grouplens. http://files.grouplens.org/datasets/hetrec2011/hetrec2011-movielens-2k-v2.zip
This was the same data that was used in the research paper. It contains many files in .dat extension.
We chose to use movie_actors, movie_countries, movie_directors, movie_genres, movie_locations, tags, movies, and tags file for our learning model. We will use user_ratedmovies later for our recommendations.

## How we converted data into dat files
The procedure to convert these files from dat to csv was to use this [stackoverflow link](https://stackoverflow.com/questions/11483920/convert-dat-files-to-either-dta-shp-bdf-or-csv) to make the files into csv. Delimiter was kept as tab.
Then the issue was of encoding. This [link](https://stackoverflow.com/questions/7256049/how-do-i-convert-an-ansi-encoded-file-to-utf-8-with-notepad) to change encoding to utf-8 with notepad by going to save as and changing encoding so pandas could read the data. There was an attempt to set encoding the file in utf-16-le [as stated here](https://stackoverflow.com/questions/43786852/python-pandas-load-csv-ansi-format-as-utf-8) but that attempt was unsuccessful hence encoding set to utf-8

In [294]:
import os

In [295]:
os.getcwd()

'C:\\Users\\Omema Ahmed\\Desktop\\Ds2 Project\\cs201-spr20-proj--cs201-s20-domination-association-Data-processing'

In [296]:
import pandas as pd
movie_actors = pd.read_csv("Data Refined\\movie_actors.csv")

In [298]:
movie_countries = pd.read_csv("Data Refined\\movie_countries.csv")

In [300]:
movie_directors = pd.read_csv("Data Refined\\movie_directors.csv")

In [302]:
movie_genres = pd.read_csv("Data Refined\\movie_genres.csv")

In [304]:
movie_locations = pd.read_csv("Data Refined\\movie_locations.csv")

In [414]:
movie_tags = pd.read_csv("Data Refined\\movie_tags.csv")

In [308]:
movies = pd.read_csv("Data Refined\\movies.csv")

In [310]:
tags = pd.read_csv("Data Refined\\tags.csv")

In [424]:
#discarding unnecessary columns
final_table = movies[movies.columns[:6]]

#removing imdb ID and picture URL, and spanishTitle column
final_table= final_table.drop(["imdbID", "spanishTitle", "imdbPictureURL"], axis=1)

#merging countries table with movies
final_table = final_table.merge(movie_countries,left_on="id", right_on="movieID")

#merging directors table with movies
final_table = final_table.merge(movie_directors,left_on="id", right_on="movieID")

#merging tags table with movies
final_table = final_table.merge(tags, on ="id")

#removing unnecessary columns
final_table= final_table.drop(["movieID_x", "movieID_y", "directorID"], axis=1)

#creating dictionaries to store genres, actors and locations against movieIDs
genres_dict, actors_dict, loc_dict ={}, {}, {}

for i in movie_genres.iterrows():
    if i[1][0] not in genres_dict:
        genres_dict[i[1][0]] = []
    genres_dict[i[1][0]].append(i[1][1])

for i in movie_actors.iterrows():
    if i[1][0] not in actors_dict:
        actors_dict[i[1][0]] = []
    actors_dict[i[1][0]].append(i[1][2])

for i in movie_locations.iterrows():
    if i[1][0] not in loc_dict:
        loc_dict[i[1][0]] = []
    for x in range(1,4):
        if i[1][x] not in loc_dict[i[1][0]] and isinstance(i[1][x],str):
            loc_dict[i[1][0]].append(i[1][x])
# print(loc_dict)

In [421]:
#creating dictionary to store all movie details against movieIDs
final_dict = {}
for i in final_table.iterrows():
    if i[1][0] not in final_dict:
        final_dict[i[1][0]] = []
    #adding title, year, country and director
    for x in range(1,6):
        final_dict[i[1][0]].append(i[1][x])
    #adding genres
    if i[1][0] in genres_dict and len(genres_dict[i[1][0]]) != 0:
        for val in genres_dict[i[1][0]]:
            final_dict[i[1][0]].append(val)
    #adding actors
    if i[1][0] in actors_dict and len(actors_dict[i[1][0]]) != 0:
        for val in actors_dict[i[1][0]]:
            final_dict[i[1][0]].append(val)
    #adding locations
    if i[1][0] in loc_dict and len(loc_dict[i[1][0]]) != 0:
        for val in loc_dict[i[1][0]]:
            final_dict[i[1][0]].append(val)
# for i in range(1,5):
#     print(final_dict[i])

['Toy story', 1995, 'USA', 'John Lasseter', 'earth', 'Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy', 'Annie Potts', 'Bill Farmer', 'Don Rickles', 'Erik von Detten', 'Greg Berg', 'Jack Angel', 'Jan Rabson', 'Jim Varney', 'Joan Cusack', 'Joe Ranft', 'John Morris', 'John Ratzenberger', 'Kendall Cunningham', 'Laurie Metcalf', 'Patrick Pinney', 'Penn Jillette', 'Philip Proctor', 'R. Lee Ermey', 'Sarah Freeman', 'Scott McAfee', 'Sherry Lynn', 'Tim Allen', 'Tom Hanks', 'Wallace Shawn']
['Jumanji', 1995, 'USA', 'Joe Johnston', 'police', 'Adventure', 'Children', 'Fantasy', 'Peter Bryant', 'Adam Hann-Byrd', 'Bebe Neuwirth', 'Bonnie Hunt', 'Bradley Pierce', 'Darryl Henriques', 'David Alan Grier', 'David Hyde Pierce', 'Gillian Barber', 'James Handy', 'Jonathan Hyde', 'Kirsten Dunst', 'Laura Bell Bundy', 'Lloyd Berry', 'Malcolm Stewart', 'Patricia Clarkson', 'Robin Driscoll', 'Robin Williams', 'Canada', 'British Columbia', 'Delta', 'Maple Ridge', 'Vancouver', 'USA', 'Maine', 'Kennebunk',

In [425]:
import math
import bitarray
import mmh3

class bloomFilter():
    def __init__(self, total):        
        #calculates false positive probability
        # self.fp = self.get_fp()
        self.fp = 0.001

        #calculates the size of bit array
        self.bits = self.get_size(total, self.fp)
        # print(self.bits)

        #calculates optimum numb of hash function to use
        self.totalhash = self.get_total_hash(self.bits, total)
        # print(self.totalhash)

        #creates a bit array of the given size
        self.bitarray = bitarray.bitarray(self.bits)

        #initializes all bits to 0
        self.bitarray.setall(0)

        #total items added
        self.items = 0

    def get_fp(self):
        return (1- math.exp(-(self.totalhash * self.items) / self.bits)) ** self.totalhash

    def add(self, element):
        for i in self.element_index(element):
            self.bitarray[i] = 1
        self.items += 1 

    def check(self, element):
        for i in self.element_index(element):
            if self.bitarray[i] == 0:
                return False
        return True

    def get_size(self, total, fp):
        m = -(total* math.log(fp))/(math.log(2)**2)
        return round(m) 

    def get_total_hash(self, bits, total):
        k = (bits/total) * math.log(2)
        return round(k)        

    def hash_index(self, hash):
        return hash % self.bits
    
    def element_index(self, element):
        return [self.hash_index(mmh3.hash(element, i))for i in range(self.totalhash)]

In [426]:
bloom_filters = {}
for i in final_dict:
    if final_dict[i][0] not in bloom_filters:
        obj = bloomFilter(237)
        for x in final_dict[i]:
            obj.add(str(x))
        bloom_filters[final_dict[i][0]] = obj.bitarray
        break
print(bloom_filters)

{'Toy story': bitarray('1000000000000000000000000000000000100000010000000001000000000000000000000100000001000000001010000100001000001001000000000000000100000000000000011000001001100010000000100000000000001000001000000010010100100000000100000000000000001000010100001001010000000000100000110000000001000000101000000000000000000000000101100000001000000000000010000000000000000001000100000000000000000000000000000000000000001000000001000000000000000010000000000100000010000000000000000000010001010000010001001000000000000000000000000000001000000000000000001000000000000000000000010000000010000000100000000000001000001010100010000000000000010000000100000000000000100001001000001000100000000001000000100000000000100000000000100001000000000110010000000000001010001000000000000000000000000100001000000000000000000000001000000000000000010000000000000100100000000000000100011000000000000001000000000000000000000000010000010000010000000001000000000000000000110001000000101000000000100000000000000000000000000010